# Análisis Exporatorio de los datos

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import os

### Carga y Configuración de Datos

In [2]:
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Rutas relativas
BASE_DIR = "../../data/processed/tlc_clean"
ANIO_ANALISIS = "2025" # Se puede cambiar

In [5]:
# --- FUNCIÓN DE CARGA ---
def cargar_datos(tipo_vehiculo, anio, muestra=0.1):
    """
    Carga los datos de la carpeta específica (yellow o fhvhv).
    Muestra: 0.1 carga el 10% de los datos para ir rápido.
    """
    ruta_busqueda = os.path.join(BASE_DIR, tipo_vehiculo, anio, "*.parquet")
    archivos = sorted(glob.glob(ruta_busqueda))
    
    if not archivos:
        print(f" No se han encontrado archivos en: {ruta_busqueda}")
        return pd.DataFrame() # Devuelve vacío si falla
    
    print(f" Cargando {len(archivos)} archivos de {tipo_vehiculo} ({anio})...")
    
    dfs = []
    for f in archivos:
        try:
            df_temp = pd.read_parquet(f)
            # Cogemos una muestra aleatoria para no saturar la RAM
            if muestra < 1.0:
                df_temp = df_temp.sample(frac=muestra, random_state=42)
            dfs.append(df_temp)
        except Exception as e:
            print(f"Error leyendo {f}: {e}")
            
    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        df_final['tipo_vehiculo'] = tipo_vehiculo
        
        # --- ORDENAR CRONOLÓGICAMENTE ---
        if 'fecha_inicio' in df_final.columns:
            df_final.sort_values('fecha_inicio', inplace=True)
            
        return df_final
    return pd.DataFrame()

In [6]:
# --- EJECUCIÓN DE CARGA ---
df_yellow = cargar_datos("yellow", ANIO_ANALISIS, muestra=0.15)
df_vtc = cargar_datos("fhvhv", ANIO_ANALISIS, muestra=0.15)      

# Unimos todo (dataframe global)
df = pd.concat([df_yellow, df_vtc], ignore_index=True)

if 'fecha_inicio' in df.columns:
    print("Ordenando dataset global por fecha...")
    df.sort_values('fecha_inicio', inplace=True)

print(f"\n {len(df):,} viajes cargados.")
print(df.head(10))

 Cargando 11 archivos de yellow (2025)...
 Cargando 11 archivos de fhvhv (2025)...
Ordenando dataset global por fecha...

 39,413,892 viajes cargados.
               fecha_inicio           fecha_fin  origen_id  destino_id  \
0       2009-01-01 00:05:50 2009-01-01 00:10:56        239          50   
1       2009-01-01 14:08:04 2009-01-01 14:45:04        138         142   
2       2024-12-31 20:54:50 2024-12-31 21:30:18        246          48   
3       2024-12-31 23:48:48 2024-12-31 23:55:13        114         148   
6252859 2025-01-01 00:00:00 2025-01-01 00:30:04        247          81   
6252860 2025-01-01 00:00:02 2025-01-01 00:27:02         37         165   
6252861 2025-01-01 00:00:03 2025-01-01 00:09:47        231         186   
6252862 2025-01-01 00:00:03 2025-01-01 00:13:13         85          61   
6252863 2025-01-01 00:00:04 2025-01-01 00:05:15        173          56   
6252864 2025-01-01 00:00:04 2025-01-01 00:07:08        241         136   

         distancia     precio  dur